In [1]:
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import zipfile
from IPython.display import display, Image
from scipy import ndimage
from sklearn.linear_model import LogisticRegression
from six.moves.urllib.request import urlretrieve
from six.moves import cPickle as pickle

from skimage import color, io
from scipy.misc import imresize

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D, Activation, GlobalAveragePooling2D
from keras.layers import merge, Input, Lambda
from keras.callbacks import EarlyStopping
from keras.models import Model

np.random.seed(31337)

# Config the matplotlib backend as plotting inline in IPython
%matplotlib inline

Using TensorFlow backend.


In [2]:
%%time
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np

resnet_codes_model = ResNet50(input_shape=(300,300,3), include_top=False, weights='imagenet')    
resnet_codes_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 300, 300, 3)   0                                            
____________________________________________________________________________________________________
zeropadding2d_1 (ZeroPadding2D)  (None, 306, 306, 3)   0           input_1[0][0]                    
____________________________________________________________________________________________________
conv1 (Convolution2D)            (None, 150, 150, 64)  9472        zeropadding2d_1[0][0]            
____________________________________________________________________________________________________
bn_conv1 (BatchNormalization)    (None, 150, 150, 64)  256         conv1[0][0]                      
___________________________________________________________________________________________

In [3]:
# l=resnet_codes_model.layers[175]
# l.get_config()

In [4]:
# # trim after fire9/concat
# model = Model(input=resnet_codes_model.inputs, output=resnet_codes_model.layers[175].output)
# model.summary()
# model.input_shape[1:3], model.output_shape

# Final model
model=Model(input=resnet_codes_model.input, output=GlobalAveragePooling2D()(resnet_codes_model.output))

In [5]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 300, 300, 3)   0                                            
____________________________________________________________________________________________________
zeropadding2d_1 (ZeroPadding2D)  (None, 306, 306, 3)   0           input_1[0][0]                    
____________________________________________________________________________________________________
conv1 (Convolution2D)            (None, 150, 150, 64)  9472        zeropadding2d_1[0][0]            
____________________________________________________________________________________________________
bn_conv1 (BatchNormalization)    (None, 150, 150, 64)  256         conv1[0][0]                      
___________________________________________________________________________________________

In [20]:
from keras.preprocessing.image import ImageDataGenerator

def img_to_bgr(im):
    return im[:, :, ::-1]

datagen = ImageDataGenerator(rescale=1., featurewise_center=True,preprocessing_function=img_to_bgr) #(rescale=1./255)
datagen.mean=np.array([103.939, 116.779, 123.68],dtype=np.float32).reshape(1,1,3)

In [26]:
train_batches = datagen.flow_from_directory("train", model.input_shape[1:3], shuffle=False, batch_size=32,save_to_dir="aug")
valid_batches = datagen.flow_from_directory("valid", model.input_shape[1:3], shuffle=False, batch_size=32)
test_batches = datagen.flow_from_directory("test", model.input_shape[1:3], shuffle=False, batch_size=32, class_mode=None)
model.input_shape[1:3]

Found 22949 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.


(300, 300)

In [8]:

train_codes = model.predict_generator(train_batches, train_batches.nb_sample)

In [9]:

valid_codes = model.predict_generator(valid_batches, valid_batches.nb_sample)

In [10]:

test_codes = model.predict_generator(test_batches, test_batches.nb_sample)

In [16]:
import h5py

from keras.utils.np_utils import to_categorical

with h5py.File("ResNet50-300x300_codes-train.h5") as hf:
    hf.create_dataset("X_train", data=train_codes)
    hf.create_dataset("X_valid", data=valid_codes)
    hf.create_dataset("Y_train", data=to_categorical(train_batches.classes))
    hf.create_dataset("Y_valid", data=to_categorical(valid_batches.classes))

with h5py.File("ResNet50-300x300_codes-test.h5") as hf:
    hf.create_dataset("X_test", data=test_codes)



In [ ]:

# train_preds=squeezenet_model.predict(X_hard, verbose=0)
# print(train_preds.shape)

# pickle_file = 'squeezenet_hard_preds-86528D.npy'
# np.save(pickle_file,train_preds)
    
# statinfo = os.stat(pickle_file)
# print('File size:', statinfo.st_size)

In [27]:
model.predict_generator(train_batches, 32)

array([[ 0.1278301 ,  0.04724548,  0.        , ...,  0.        ,
         0.19625981,  0.10717522],
       [ 0.        ,  0.23558475,  0.03428514, ...,  0.10260688,
         0.01927472,  0.03664623],
       [ 0.01790496,  0.42267621,  0.14346312, ...,  0.2020939 ,
         0.66600299,  0.09572547],
       ..., 
       [ 0.01981861,  0.63298512,  0.00554222, ...,  0.04417805,
         0.24147435,  0.6002565 ],
       [ 0.0926577 ,  0.75804758,  0.48668495, ...,  0.10559805,
         0.68690193,  0.13546729],
       [ 0.08933438,  0.10526311,  0.        , ...,  0.50785154,
         0.11918554,  0.00860771]], dtype=float32)

AttributeError: 'module' object has no attribute 'close'